<a href="https://colab.research.google.com/github/duongtrung/Pytorch-tutorials/blob/main/12_pytorch_detectron2_object_detection_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Brief introduction

official Git: [detectron2](https://github.com/facebookresearch/detectron2)

Detectron2 is Facebook AI Research's next generation library that provides state-of-the-art detection and segmentation algorithms. It is the successor of Detectron and maskrcnn-benchmark. It supports a number of computer vision research projects and production applications in Facebook.

<img src="resources/Detectron2-Logo-Horz.png" width="500">

In [1]:
# install detectron2
# conda install -c conda-forge detectron2

# By the time of this tutorial (09.2022) the Detectron2 does not officially support Windows OS.
# It is a git tricky to install detectron2

# For windows user, read this blog
# https://medium.com/@yogeshkumarpilli/how-to-install-detectron2-on-windows-10-or-11-2021-aug-with-the-latest-build-v0-5-c7333909676f

# Note from my installation
# Microsoft Visual C++ 14.0 or greater is required: https://visualstudio.microsoft.com/visual-cpp-build-tools/
# Do not install Visual Studio 2022. Try either 2017 or 2019 version. I tried Visual Studio 2019 version 16.11.19. https://learn.microsoft.com/en-us/visualstudio/releases/2019/release-notes
# I have RTX 3080, so I install CUDA Toolkits 11.3.
# If you get nvcc.exe' failed with exit code 1, do the following steps:
# - Edit PATH to where nvcc.exe (CUDA 11.3) is. For example, I copy to C:\CUDA\v11.3\bin
# - Open the detectron2\layers\csrc\nms_rotated\nms_rotated_cuda.cu file, and comment the two ifdef statements. See https://github.com/facebookresearch/detectron2/issues/1601

In [4]:
# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2

# get image
#!wget http://images.cocodataset.org/val2017/000000439715.jpg -O input.jpg
im = cv2.imread("resources/000000439715.jpg")

# Create config
cfg = get_cfg()
cfg.merge_from_file("detectron2/configs/COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.WEIGHTS = "D:/gdrive/detectron2/faster_rcnn_R_101_FPN_3x/137851257/model_final_68b088.pkl"

In [5]:
# Create predictor
predictor = DefaultPredictor(cfg)

# Make prediction
outputs = predictor(im)

AssertionError: Checkpoint D:/gdrive/detectron2/faster_rcnn_R_101_FPN_3x/137851257/model_final_68b088.pkl not found!